<a href="https://colab.research.google.com/github/Sandell0/Colab-experiments/blob/master/Learn_sha_and_back_using_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
str1 = "1234567812345678123456781234567812345678123456781234567812345678"
str2 = "GeeasfsksforGeeks"

# encoding GeeksforGeeks using encode()
# then sending to SHA1()
result = hashlib.sha1(str1.encode())
#hashcode=hashlib.md5(s).hexdigest()

print(bin(int(result.hexdigest(),16)))

result = hashlib.sha1(str2.encode())
print(bin(int(result.hexdigest(),16)))


0b1000110000110110100101111010011011000001011011110010001011101110100110100111100001110001111111110110110011010000011011001010000111101000100001101000001000010110
0b11100000010110110101000011100001101110011101000001110110010110000001110001111010010010111100011110010011001011111111101111010000101010000110110111001010111110


In [ ]:
N=64
import random
import string
my_dict = {"text":[],"code":[]}
for i in range(1000000):
  custom_string = ''.join(random.choice(string.printable) for _ in range(N))
  my_dict["text"].append(custom_string)
  my_dict["code"].append(bin(int(hashlib.sha256(str2.encode()).hexdigest(),16))[2:])

In [ ]:
!pip install transformers
!pip install datasets
!pip install wandb

In [ ]:
from datasets import Dataset
data = Dataset.from_dict(my_dict)

In [ ]:
import torch
import tqdm
from datasets import load_dataset
from transformers import (
    EncoderDecoderModel,
    AutoTokenizer,
    BatchEncoding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    T5ForConditionalGeneration
)

torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained("google/byt5-base")


def tokenize_function(examples):
    tokenized_text = tokenizer(
        examples["text"],
        padding="longest",
        max_length=256,
    )
    tokenized_code = tokenizer(
        examples["code"],
        padding="longest",
        max_length=256,
    )

    examples["input_ids"] = tokenized_code["input_ids"].copy()
    examples["attention_mask"] = tokenized_code["attention_mask"].copy()
    examples["labels"] = tokenized_text["input_ids"].copy()
    examples["decoder_input_ids"] = tokenized_text["input_ids"].copy()

    #print(len(examples["input_ids"]), len(examples["decoder_input_ids"]))
    return examples


class MyCallback(TrainerCallback):
    def on_log(self, args, state, control, **kwargs):
      prompts = [
          "1000110000110110100101111010011011000001011011110010001011101110100110100111100001110001111111110110110011010000011011001010000111101000100001101000001000010110",
      ]


      # encode prompt
      encoded_prompts = tokenizer(prompts, return_tensors="pt")

      encoded_prompts = encoded_prompts.to('cuda:0')

      # generate text
      outputs = model.generate(
          input_ids=encoded_prompts["input_ids"],
          attention_mask=encoded_prompts["attention_mask"],
          max_length=256,
      )

      print(tokenizer.decode(outputs[0]).replace('\n',''))


train_test = data.train_test_split(test_size=0.001, shuffle=False)

for key in ["train", "test"]:
    train_test[key] = train_test[key].map(tokenize_function)
model = T5ForConditionalGeneration.from_pretrained('google/byt5-base') # initialize Bert2Bert from pre-trained checkpoints


training_args = TrainingArguments(
    output_dir="./sha-hacker",  # The output directory
    overwrite_output_dir=True,  # overwrite the content of the output directory
    num_train_epochs=300,  # number of training epochs
    per_device_train_batch_size=2,  # batch size for training
    per_device_eval_batch_size=2,  # batch size for evaluation
    eval_steps=100,  # 400,  # Number of update steps between two evaluations.
    save_steps=1000,  # after # steps model is saved
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=10,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    learning_rate=1e-4,
    weight_decay=0.05,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test["train"],
    eval_dataset=train_test["test"],
    callbacks=[MyCallback],
)

trainer.train()
trainer.save_model()

  0%|          | 0/999000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, code.
***** Running training *****
  Num examples = 999000
  Num Epochs = 300
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 149850000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: sandell0 (use `wandb login --relogin` to force relogin)


Step,Training Loss,Validation Loss
100,7.297400,2.346630
200,1.110200,0.098688
300,0.145900,0.039602
400,0.057100,0.011958
500,0.020400,0.000109
600,0.005300,0.000005
700,0.005500,0.000058
800,0.005200,0.003949
900,0.003500,0.000001
1000,0.001200,0.000003


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, code.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 2


...

<pad></s>
<pad>{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, code.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 2


<pad>                                                                                                                                                                                                                                                             
<pad>{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{{


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, code.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 2


<pad>ھbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
